<a href="https://colab.research.google.com/github/mbagci06/lung_covid/blob/main/Unet_train_lung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# BASICS 
import os
import numpy as np
import torch

# Colab 
from pydrive.drive import GoogleDrive
from google.colab import drive
from google.colab import files
from pydrive.auth import GoogleAuth
from google.colab import auth
import torchvision.transforms as transforms
from oauth2client.client import GoogleCredentials
# to reach the dataset in the drive files and other functions but submittion there should not be
drive.mount('/content/drive')
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# MODEL and TRAIN
import torchvision
import torchvision.models
from torch.utils.data import DataLoader,Dataset
import random
from skimage import io
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import pyplot as plt
from torch import optim
from PIL import Image

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# The scripts contains some important functions have copied from the drive files in to the our script
!cp /content/drive/Shareddrives/FurkanDataandModels/Furkan_covid/codes/functions.ipynb  /content
!cp -R /content/drive/Shareddrives/FurkanDataandModels/Furkan_covid/codes/model /content

In [ ]:
# running  .ipynb scripts to call their functions
%run functions.ipynb
from model.unet import UNet, PretrainedUNet

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# The Calling the validaton dataset
val=Lung_Dataset(split = "Val",class_folder='covid',shuffle=True,sample_size=300)# The length of the validation dataset to calculate the average value of the loss and accuracy
# The size of the dataset for calculate average datas
val_size=val.__len__()
# The Train dataset 
tra=Lung_Dataset(split = "Train",class_folder='covid',shuffle=True,sample_size=2000)
tra_size=tra.__len__()
# # The transform functions to conver the PIL images to the tensors


# The device looks for GPU if cannot find goes with CPU
device=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# The model is called by this function from the model.py file 
net = PretrainedUNet( in_channels=1, out_channels=2,batch_norm=True).to(device)
# The adam optimizer for optimization the values will be same for the other model training
optimizer = optim.Adam(net.parameters(), lr= 0.0001)
# The Loss function Cross entropy the value 
criterion = nn.CrossEntropyLoss()


In [ ]:
batch_size=25
# loading images and masks for mini batches
data_loader = torch.utils.data.DataLoader(
        tra, batch_size=batch_size,  num_workers=2, drop_last=True)

data_loader_val = torch.utils.data.DataLoader(
        val, batch_size=batch_size,  num_workers=2, drop_last=True)

In [ ]:
# The stiation of the GPU
!nvidia-smi

Wed May 25 19:52:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    26W /  70W |   1360MiB / 15109MiB |      6%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# The number of the epoch 
epoch_number=70
# list for the calculating validation Loss
val_loss_array=[]
# list for the calculating train Loss
train_loss_array=[]
# list for the calculating validation accuracy
acc_val_array=[]
# list for the calculating train accuracy
acc_train_array=[]
for t in range(epoch_number):
    # reset the loss and the accuracy values each epoch 
    val_loss=0
    train_loss=0
    acc_val=0
    acc_train=0
    for batch_idx, sample in enumerate(data_loader):
        # Train data preparing 
        imgs , true_masks = sample['image'],sample['mask']
        imgs = imgs.to(device=device)
        # The negative takes the masks and doubles the channels with inverse (0 to 1) shape  
        true_masks=Negative_mask(true_masks,imgs.size(dim=0))
        true_masks = true_masks.to(device=device)
        net.train()
        # zero the parameter gradients
        optimizer.zero_grad()
        

        with torch.set_grad_enabled(True):
            # Train data evaluation
            masks_pred = net(imgs)
            loss = criterion(masks_pred, true_masks)
            train_loss += loss.item()
            # Updating the params
            loss.backward()
            optimizer.step()
            # Acc calc train
            acc_train+=dice( masks_pred, true_masks)
    for batch_idx, sample in enumerate(data_loader_val):
        # Validation data preparing  and calling 
        # calcel gradient feature 
        net.eval()

        # Call iamge and mask
        imgs , true_masks = sample['image'],sample['mask']

        imgs = imgs.to(device=device)
        true_masks=Negative_mask(true_masks,imgs.size(dim=0))
        true_masks = true_masks.to(device=device)

        # zero the parameter gradients
        optimizer.zero_grad()

        with torch.set_grad_enabled(False):
            # Validation evaluation 
            masks_pred = net(imgs)
            masks_pred = masks_pred.to(device=device)
            loss_val = criterion(masks_pred, true_masks)
            acc_val +=dice( masks_pred, true_masks)
            val_loss += loss_val.item()

    batch_size
    print('Epoch :',t+1,'/',str(epoch_number))
    print('Validation Loss:v',(val_loss)/(val_size/batch_size))
    print('Validation Acc:',(acc_val)/(val_size/batch_size))
    print('Train Acc:',(acc_train)/(tra_size/batch_size))
    print('Train  Loss   :t',(train_loss)/(tra_size/batch_size))
    # Store epoch progress
    val_loss_array.append((val_loss)/(val_size/batch_size))
    train_loss_array.append((train_loss)/(tra_size/batch_size))
    acc_val_array.append((acc_val)/(val_size/batch_size))
    acc_train_array.append((acc_train)/(tra_size/batch_size))

Epoch : 1 / 70
Validation Loss:v 0.3452342947324117
Validation Acc: 0.15601764929791292
Train Acc: 0.08252427425468341
Train  Loss   :t 0.39961692318320274
Epoch : 2 / 70
Validation Loss:v 0.29208548615376156
Validation Acc: 0.45109232763449353
Train Acc: 0.28768910840153694
Train  Loss   :t 0.2972316861152649
Epoch : 3 / 70
Validation Loss:v 0.25454239671428996
Validation Acc: 0.637824684381485
Train Acc: 0.619723304733634
Train  Loss   :t 0.2608377233147621
Epoch : 4 / 70
Validation Loss:v 0.25598656634489697
Validation Acc: 0.728227565685908
Train Acc: 0.7133053824305534
Train  Loss   :t 0.23066591322422028
Epoch : 5 / 70
Validation Loss:v 0.2135783148308595
Validation Acc: 0.7973591933647791
Train Acc: 0.7784204758703709
Train  Loss   :t 0.2058021554723382
Epoch : 6 / 70
Validation Loss:v 0.20332962398727736
Validation Acc: 0.9046633988618851
Train Acc: 0.8572256036102772
Train  Loss   :t 0.1885024918243289
Epoch : 7 / 70
Validation Loss:v 0.19395970553159714
Validation Acc: 0.9197

In [ ]:
train_loss

1.2343059601262212

In [ ]:
import copy
# Save Model to the drive folder 
best_model_wts = copy.deepcopy(net.state_dict())
torch.save({    
    'epoch':epoch_number,
    'model_state_dict':best_model_wts,
    'optimizer_state_dict': optimizer.state_dict()
     
},os.path.join(os.getcwd(),'/content/drive/Shareddrives/FurkanDataandModels/Furkan_covid/codes/model_covid_pre_{}_epoch.pt'.format(epoch_number)))

In [ ]:
# Save loss and acc information to the txt 
textfile = open("/content/drive/Shareddrives/FurkanDataandModels/Furkan_covid/codes/model_covid_pre_{}_epo.txt".format(epoch_number), "w")
# The form of the saved values are important when we used them to plot
biglist=val_loss_array+   train_loss_array+    acc_val_array+    acc_train_array
textfile. write("running_loss_array+   train_loss_array+    acc_val_array+    acc_train_array"+ "\n")
for element in [biglist]:
    textfile. write(str(element) + "\n")
textfile. close()
